In [1]:
import cv2

thres = 0.5

def calculate_distance(box):
    focal_length = 1000
    known_object_width = 10
    x, y, w, h = box
    distance = (known_object_width * focal_length) / w
    return distance

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
classNames = []

config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'frozen_inference_graph.pb'

net = cv2.dnn_DetectionModel(weightsPath, config_file)
net.setInputSize(320, 320)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

while True:
    success, img = cap.read()
    classIds, confs, bbox = net.detect(img, confThreshold=thres)
    
    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
            distance = calculate_distance(box)
            if distance < 25:
                color = (0, 0, 255)
            else:
                color = (0, 255, 0)
            cv2.rectangle(img, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), color, thickness=2)
            cv2.putText(img, f'Distance: {distance:.2f} cm', (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("Output", img)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()
